In [60]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import string 
import seaborn as sns 
import nltk 
import spacy
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import os 
import sys 
import contractions 

In [61]:
path = "D:/InterviewAutomation/data"

In [63]:
excel_files = [file for file in os.listdir(path) if file.endswith(('.xls', '.xlsx'))]

In [64]:
for file in excel_files:
    df = pd.read_excel(path+'/'+file)
    print(df.columns)

Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
       'Reason for decision', 'Job Description'],
      dtype='object')
Index(['ID', 'Name', 'Role', 'Transcript'

In [5]:
# # Changing the Columns of the 4th dataset as it does have the Wrong Columns
# df = pd.read_excel(excel_files[3])
# df.columns = ['ID', 'Name', 'Role', 'Transcript', 'Resume', 'decision',
#        'Reason for decision', 'Job Description']
# df.to_excel(path+'/'+'dataset4.xlsx',index=False)

In [6]:
# for file in excel_files:
#     df = pd.read_excel(path+'/'+file)
#     print(df.columns)

In [65]:
# Code for Merging all the datasets and creating the full dataset for traing and testing
dataframes = []

for file in excel_files:
    file_path = os.path.join(path, file)
    df = pd.read_excel(file_path)
    dataframes.append(df)

# Combine all dataframes into a single dataframe
data = pd.concat(dataframes, ignore_index=True)

In [66]:
data.shape

(3250, 8)

In [67]:
data.head()

,ID,Name,Role,Transcript,Resume,decision,Reason for decision,Job Description
0,brenbr359,Brent Brown,Product Manager,Product Manager Interview Transcript\n\nInterv...,Here's a sample resume for Brent Brown applyin...,select,Experience,We are looking for a skilled Product Manager w...
1,jameay305,James Ayala,Software Engineer,Software Engineer Interview Transcript\n\nInte...,Here's a sample resume for James Ayala applyin...,select,Experience,We are looking for a skilled Software Engineer...
2,scotri565,Scott Rivera,Data Engineer,Here is a simulated interview for Scott Rivera...,Here's a sample resume for Scott Rivera applyi...,reject,Experience,We are looking for a skilled Data Engineer wit...
3,emilke232,Emily Kelly,UI Engineer,Interview Transcript: Emily Kelly for UI Engin...,Here's a sample resume for Emily Kelly:\n\nEmi...,select,Experience,We are looking for a skilled UI Engineer with ...
4,ashlra638,Ashley Ray,Data Scientist,Data Scientist Interview Transcript\n\nCompany...,Here's a sample resume for Ashley Ray applying...,reject,Cultural Fit,We are looking for a skilled Data Scientist wi...


In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3250 entries, 0 to 3249
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   3250 non-null   object
 1   Name                 3250 non-null   object
 2   Role                 3250 non-null   object
 3   Transcript           3250 non-null   object
 4   Resume               3250 non-null   object
 5   decision             3250 non-null   object
 6   Reason for decision  3250 non-null   object
 7   Job Description      3250 non-null   object
dtypes: object(8)
memory usage: 203.2+ KB


In [69]:
for col in data.columns:
    data[col] = data[col].apply(lambda x: x.lower())

In [70]:
data['decision'].unique()

array(['select', 'reject', 'rejected', 'selected'], dtype=object)

In [71]:
def process_decision(text):
    if text in ['select','selected']:
        return 'select'
    else :
        return 'reject'

In [72]:
data['decision'] = data['decision'].apply(process_decision )

In [73]:
data['Role'].unique()

array(['product manager', 'software engineer', 'data engineer',
       'ui engineer', 'data scientist', 'data analyst', 'ui designer',
       'digital marketing specialist', 'cybersecurity specialist',
       'hr specialist', 'network engineer', 'graphic designer',
       'game developer', 'mobile app developer', 'cloud architect',
       'business analyst', 'database administrator', 'devops engineer',
       'machine learning engineer', 'content writer',
       'system administrator', 'ui/ux designer', 'ai engineer',
       'project manager', 'software developer'], dtype=object)

In [74]:
unique_count = data.groupby('Role')['ID'].count()
unique_count

Role
ai engineer                      13
business analyst                 11
cloud architect                   8
content writer                    5
cybersecurity specialist          9
data analyst                    267
data engineer                   509
data scientist                  592
database administrator            9
devops engineer                  16
digital marketing specialist     14
game developer                   10
graphic designer                 15
hr specialist                    12
machine learning engineer        13
mobile app developer              7
network engineer                 16
product manager                 514
project manager                  56
software developer               66
software engineer               551
system administrator              5
ui designer                     234
ui engineer                     291
ui/ux designer                    7
Name: ID, dtype: int64

In [75]:
## Dropping Rows which have count < 100 
for count , role in zip(unique_count.values, unique_count.index):
    if count < 100 :
        data = data[data['Role'] != role] 

In [76]:
unique_count = data.groupby('Role')['ID'].count()
unique_count

Role
data analyst         267
data engineer        509
data scientist       592
product manager      514
software engineer    551
ui designer          234
ui engineer          291
Name: ID, dtype: int64

In [77]:
data.shape

(2958, 8)

In [78]:
## Performing Feature Engineering
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [81]:
nlp = spacy.load("en_core_web_sm") 
stopword_s = stopwords.words('english')
lemmatizer = WordNetLemmatizer() 

def textpreprocessing(text):
    text = contractions.fix(text) # expanding the contractions like , I'm with I am and it's with It is and so on...
    text = re.sub(r'[!"#$%&\'()*+,-./:;<=>?@\[\]^_`{|}~]', '', text)  # Removing all these characters from the text 
    sentences = re.split(r'(?<=[.!?])\s+', text)   # Splitting the text as sentences using the regular expressions     
    for idx, sent in enumerate(sentences):
        words = nlp(sent)  # converting the sentence into words
        words = [word.text for word in words if word.text.lower() not in stopword_s] # Removing the stopwords from the text
        words = [lemmatizer.lemmatize(word) for word in words]  # Using the Lemmatization techniques
        sentences[idx] = ' '.join(words)  # Replacing the original sentence with the new sentence 
    return ' '.join(sentences)  # Join the sentences back together  

In [83]:
data['Transcript_processed'] = data['Transcript'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.xlsx')

In [84]:
data['Resume_processed'] = data['Resume'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.xlsx')

In [85]:
data['Job_Description_processed'] = data['Job Description'].apply(textpreprocessing)
# Saving the processed Data to a temporary file
data.to_excel(path+'/'+'final_data.xlsx')

In [86]:
path_final = "D:/InterviewAutomation/data/final_data.xlsx"

In [87]:
import pandas as pd

In [88]:
df = pd.read_excel(path_final)

In [90]:
df.head()

,Unnamed: 0,ID,Name,Role,Transcript,Resume,decision,Reason for decision,Job Description,Transcript_processed,Resume_processed,Job_Description_processed
0,0,brenbr359,brent brown,product manager,product manager interview transcript\n\ninterv...,here's a sample resume for brent brown applyin...,select,experience,we are looking for a skilled product manager w...,product manager interview transcript \n\n inte...,sample resume brent brown applying role produc...,looking skilled product manager expertise data...
1,1,jameay305,james ayala,software engineer,software engineer interview transcript\n\ninte...,here's a sample resume for james ayala applyin...,select,experience,we are looking for a skilled software engineer...,software engineer interview transcript \n\n in...,sample resume james ayala applying software en...,looking skilled software engineer expertise py...
2,2,scotri565,scott rivera,data engineer,here is a simulated interview for scott rivera...,here's a sample resume for scott rivera applyi...,reject,experience,we are looking for a skilled data engineer wit...,simulated interview scott rivera applying role...,sample resume scott rivera applying role data ...,looking skilled data engineer expertise cloud ...
3,3,emilke232,emily kelly,ui engineer,interview transcript: emily kelly for ui engin...,here's a sample resume for emily kelly:\n\nemi...,select,experience,we are looking for a skilled ui engineer with ...,interview transcript emily kelly ui engineer p...,sample resume emily kelly \n\n emily kelly \n ...,looking skilled ui engineer expertise uiux des...
4,4,ashlra638,ashley ray,data scientist,data scientist interview transcript\n\ncompany...,here's a sample resume for ashley ray applying...,reject,cultural fit,we are looking for a skilled data scientist wi...,data scientist interview transcript \n\n compa...,sample resume ashley ray applying data scienti...,looking skilled data scientist expertise sql m...
